In [1]:
library(TxDb.Hsapiens.UCSC.hg38.knownGene)

Loading required package: GenomicFeatures
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, grep, grepl,
    intersect, is.unsorted, lapply, lengths, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unlist, unsplit

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: IRange

In [2]:
exbyg <- exonsBy(TxDb.Hsapiens.UCSC.hg38.knownGene, "gene")

In [3]:
exns <- exons(TxDb.Hsapiens.UCSC.hg38.knownGene)

In [4]:
head(exbyg)

GRangesList object of length 6:
$1 
GRanges object with 15 ranges and 2 metadata columns:
       seqnames               ranges strand   |   exon_id   exon_name
          <Rle>            <IRanges>  <Rle>   | <integer> <character>
   [1]    chr19 [58346806, 58347029]      -   |    264625        <NA>
   [2]    chr19 [58347353, 58347640]      -   |    264626        <NA>
   [3]    chr19 [58348466, 58349128]      -   |    264627        <NA>
   [4]    chr19 [58349568, 58350651]      -   |    264628        <NA>
   [5]    chr19 [58350370, 58350651]      -   |    264629        <NA>
   ...      ...                  ...    ... ...       ...         ...
  [11]    chr19 [58357585, 58357649]      -   |    264636        <NA>
  [12]    chr19 [58357952, 58358286]      -   |    264637        <NA>
  [13]    chr19 [58358489, 58358585]      -   |    264638        <NA>
  [14]    chr19 [58359197, 58359323]      -   |    264639        <NA>
  [15]    chr19 [58362677, 58362848]      -   |    264640        <NA>


In [5]:
tsgs <- read.table("~/Desktop/test.txt")
tsgs <- tsgs[2:nrow(tsgs),]
names(tsgs) <- c("GeneID","GeneSymbol")

In [6]:
exns$istsg=0

In [7]:
head(exns)

GRanges object with 6 ranges and 2 metadata columns:
      seqnames         ranges strand |   exon_id     istsg
         <Rle>      <IRanges>  <Rle> | <integer> <numeric>
  [1]     chr1 [11874, 12227]      + |         1         0
  [2]     chr1 [12595, 12721]      + |         2         0
  [3]     chr1 [12613, 12721]      + |         3         0
  [4]     chr1 [12646, 12697]      + |         4         0
  [5]     chr1 [13221, 14409]      + |         5         0
  [6]     chr1 [13403, 14409]      + |         6         0
  -------
  seqinfo: 455 sequences (1 circular) from hg38 genome

In [8]:
for (i in tsgs$GeneID){ # Loop through tumor suppressor gene IDs
    
    tmp <- match(exbyg[[i]]$exon_id,exns$exon_id) # Get indices of exons
    if (length(tmp)==0) {
        next
    }
    
    exns[tmp]$istsg <- 1

}

In [9]:
head(exns[exns$istsg==1])

GRanges object with 6 ranges and 2 metadata columns:
      seqnames             ranges strand |   exon_id     istsg
         <Rle>          <IRanges>  <Rle> | <integer> <numeric>
  [1]     chr1 [1013467, 1013576]      + |        88         1
  [2]     chr1 [1013984, 1014539]      + |        89         1
  [3]     chr1 [1167104, 1167198]      + |       141         1
  [4]     chr1 [1167863, 1167952]      + |       142         1
  [5]     chr1 [3652565, 3652641]      + |       473         1
  [6]     chr1 [3682333, 3682430]      + |       474         1
  -------
  seqinfo: 455 sequences (1 circular) from hg38 genome

In [11]:
exann<-c()
exann$chrom <- as.vector(seqnames(exns))
exann$start <- start(exns)
exann$end <- end(exns)
exann$istsg <- exns$istsg
exann <- data.frame(exann)

In [12]:
head(exann)

chrom,start,end,istsg
chr1,11874,12227,0
chr1,12595,12721,0
chr1,12613,12721,0
chr1,12646,12697,0
chr1,13221,14409,0
chr1,13403,14409,0


In [14]:
tail(exann[exann$chrom=='chrY',])

,chrom,start,end,istsg
293785,chrY,26578095,26578328,0
293786,chrY,57094007,57094084,0
293787,chrY,57094564,57094605,0
293788,chrY,57212178,57213357,0
293789,chrY,57213856,57213964,0
293790,chrY,57214350,57214703,0


In [15]:
exann <- exann[1:293790,]

In [16]:
save(exann,file='~/jackgl/sim-develop/data/exann.rda')